# 🎬 Movie Genre Classification with MiniLM + Logistic Regression

### Step 1: Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

### Step 2: Load and Explore the Dataset

In [ ]:
df = pd.read_csv("tmdb_5000_movies.csv")
df = df[['title', 'overview', 'genres']]
df = df.dropna()

# Parse the genres column
import ast
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x)[0]['name'] if ast.literal_eval(x) else None)
df = df.dropna()

df = df.rename(columns={"overview": "plot", "genres": "genre"})
df = df[['title', 'plot', 'genre']]
df.head()

### Step 3: Limit to 500 Movies for Training

In [ ]:
df = df.sample(n=500, random_state=42).reset_index(drop=True)

### Step 4: Load MiniLM Model and Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

### Step 5: Turn Plots into Vectors (Embeddings)

In [ ]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

df['embedding'] = df['plot'].progress_apply(get_embedding)

### Step 6: Train the Logistic Regression Model

In [ ]:
X = np.stack(df['embedding'].values)
y = df['genre']

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42
)

model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

### Step 7: Plot the Confusion Matrix

In [ ]:
ConfusionMatrixDisplay.from_predictions(
    y_true=y_test,
    y_pred=model.predict(X_test),
    display_labels=encoder.classes_,
    cmap='Blues',
    xticks_rotation=45
)
plt.title("MiniLM + Logistic Regression Genre Confusion Matrix")
plt.tight_layout()
plt.show()

### Step 8: Try Predicting Your Own Movie Plot!

In [ ]:
plot = input("🎬 Describe your movie plot: ")
embedding = get_embedding(plot)
prediction = encoder.inverse_transform(model.predict([embedding]))[0]
print("📢 Predicted Genre:", prediction)